In [ ]:
from ultralytics import YOLO
import cv2
import os
import numpy as np

# ---------------- CONFIG ----------------
model = YOLO("yolov8s.pt")  # or your trained model
img_dir = r"C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test_glare_fixed"
label_dir = r"C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test\labels"

out_base = "dataset/analysis"
missed_dir = os.path.join(out_base, "missed")
partial_dir = os.path.join(out_base, "partial")
success_dir = os.path.join(out_base, "success")

os.makedirs(missed_dir, exist_ok=True)
os.makedirs(partial_dir, exist_ok=True)
os.makedirs(success_dir, exist_ok=True)

IOU_THRESHOLD = 0.5
# ----------------------------------------

def load_gt_boxes(label_path, img_w, img_h):
    boxes = []
    if not os.path.exists(label_path):
        return boxes

    with open(label_path, "r") as f:
        for line in f:
            cls, x, y, w, h = map(float, line.split())
            x1 = int((x - w / 2) * img_w)
            y1 = int((y - h / 2) * img_h)
            x2 = int((x + w / 2) * img_w)
            y2 = int((y + h / 2) * img_h)
            boxes.append([x1, y1, x2, y2])
    return boxes

def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    inter = max(0, xB - xA) * max(0, yB - yA)
    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    return inter / (areaA + areaB - inter + 1e-6)

# ---------------- PROCESS ----------------
for img_name in os.listdir(img_dir):
    if not img_name.lower().endswith((".jpg", ".png")):
        continue

    img_path = os.path.join(img_dir, img_name)
    label_path = os.path.join(label_dir, img_name.replace(".jpg", ".txt").replace(".png", ".txt"))

    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    gt_boxes = load_gt_boxes(label_path, w, h)

    results = model(img_path, conf=0.25, verbose=False)[0]
    pred_boxes = results.boxes.xyxy.cpu().numpy() if results.boxes else []

    matched_gt = set()

    for p in pred_boxes:
        for i, g in enumerate(gt_boxes):
            if iou(p, g) > IOU_THRESHOLD:
                matched_gt.add(i)

    # -------- CLASSIFICATION --------
    if len(gt_boxes) > 0 and len(matched_gt) == 0:
        out_dir = missed_dir
    elif 0 < len(matched_gt) < len(gt_boxes):
        out_dir = partial_dir
    else:
        out_dir = success_dir

    # draw GT boxes (green)
    for g in gt_boxes:
        cv2.rectangle(img, (g[0], g[1]), (g[2], g[3]), (0, 255, 0), 2)

    # draw predicted boxes (red)
    for p in pred_boxes:
        cv2.rectangle(img, (int(p[0]), int(p[1])), (int(p[2]), int(p[3])), (0, 0, 255), 2)

    cv2.imwrite(os.path.join(out_dir, img_name), img)

print("Analysis images generated successfully.")


100%|██████████| 21.5M/21.5M [00:00<00:00, 62.6MB/s]


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/images/val'

Right one

In [7]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from huggingface_hub import hf_hub_download

# ==========================================================
# 1) MODELS (SCENARIO 3)
# ==========================================================
models = [
    {"name": "YOLOv8n-S3", "repo": "rubbi194/trashdetection", "file": "scenario3/yolov8n/best.pt"},
    {"name": "YOLOv8s-S3", "repo": "rubbi194/trashdetection", "file": "scenario3/yolov8s/best.pt"},
    {"name": "YOLOv8m-S3", "repo": "rubbi194/trashdetection", "file": "scenario3/yolov8m/best.pt"},
    {"name": "YOLOv8l-S3", "repo": "rubbi194/trashdetection", "file": "scenario3/yolov8l/best.pt"},
    {"name": "YOLOv8x-S3", "repo": "rubbi194/trashdetection", "file": "scenario3/yolov8x/best.pt"},
]

# ==========================================================
# 2) DATA PATHS (YOUR PATHS)
# ==========================================================
test_images_dir = r"C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test_glare_fixed"
labels_dir = r"C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test\labels"

# ==========================================================
# OUTPUT DIRECTORIES
# ==========================================================
analysis_root = "analysis"
missed_dir = os.path.join(analysis_root, "missed")
partial_dir = os.path.join(analysis_root, "partial")
success_dir = os.path.join(analysis_root, "success")
compare_dir = "comparison_multi"

os.makedirs(missed_dir, exist_ok=True)
os.makedirs(partial_dir, exist_ok=True)
os.makedirs(success_dir, exist_ok=True)
os.makedirs(compare_dir, exist_ok=True)

# ==========================================================
# DRAW BOXES WITH BANNER
# ==========================================================
def draw_boxes(img, boxes, color, label_name):
    H, W = img.shape[:2]

    banner_h = 50
    overlay = img.copy()
    cv2.rectangle(overlay, (0, 0), (W, banner_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, 0.4, img, 0.6, 0)

    cv2.putText(img, label_name, (10, 35),
                cv2.FONT_HERSHEY_SIMPLEX, 1.1, (255, 255, 255), 3)

    for b in boxes:
        cls, xc, yc, w, h = b  # always 5 now

        x1 = int((xc - w / 2) * W)
        y1 = int((yc - h / 2) * H)
        x2 = int((xc + w / 2) * W)
        y2 = int((yc + h / 2) * H)

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, str(int(cls)), (x1, max(0, y1 - 5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    return img

# ==========================================================
# POLYGON → YOLO BBOX (CRITICAL FIX)
# ==========================================================
def polygon_to_yolo_bbox(poly):
    """
    poly = [cls, x1, y1, x2, y2, ..., xn, yn]
    returns [cls, xc, yc, w, h]
    """
    cls = poly[0]
    coords = poly[1:]

    xs = coords[0::2]
    ys = coords[1::2]

    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)

    xc = (x_min + x_max) / 2
    yc = (y_min + y_max) / 2
    w = x_max - x_min
    h = y_max - y_min

    return [cls, xc, yc, w, h]

# ==========================================================
# IOU HELPERS
# ==========================================================
def yolo_to_xyxy(box, W, H):
    _, xc, yc, w, h = box
    return [
        int((xc - w/2) * W),
        int((yc - h/2) * H),
        int((xc + w/2) * W),
        int((yc + h/2) * H),
    ]

def iou(a, b):
    xA, yA = max(a[0], b[0]), max(a[1], b[1])
    xB, yB = min(a[2], b[2]), min(a[3], b[3])
    inter = max(0, xB-xA) * max(0, yB-yA)
    areaA = (a[2]-a[0]) * (a[3]-a[1])
    areaB = (b[2]-b[0]) * (b[3]-b[1])
    return inter / (areaA + areaB - inter + 1e-6)

# ==========================================================
# LOAD MODELS
# ==========================================================
loaded_models = []
for m in models:
    print("Loading:", m["name"])
    path = hf_hub_download(repo_id=m["repo"], filename=m["file"], repo_type="model")
    loaded_models.append((m["name"], YOLO(path)))

# ==========================================================
# PROCESS DATASET
# ==========================================================
IOU_THRESH = 0.5
image_exts = (".jpg", ".png", ".jpeg")

for fname in os.listdir(test_images_dir):
    if not fname.lower().endswith(image_exts):
        continue

    name = os.path.splitext(fname)[0]
    img_path = os.path.join(test_images_dir, fname)
    img = cv2.imread(img_path)
    if img is None:
        continue

    H, W = img.shape[:2]

    # ---------- LOAD GT (SEGMENTATION SAFE) ----------
    gt_boxes = []
    label_path = os.path.join(labels_dir, name + ".txt")

    if os.path.exists(label_path):
        with open(label_path) as f:
            for line in f:
                values = list(map(float, line.split()))
                if len(values) > 5:   # segmentation polygon
                    gt_boxes.append(polygon_to_yolo_bbox(values))
                else:                 # normal YOLO box
                    gt_boxes.append(values)

    gt_panel = draw_boxes(img.copy(), gt_boxes, (0,255,0), "GROUND TRUTH")
    panels = [gt_panel]

    # ---------- PER MODEL ----------
    for model_name, model in loaded_models:
        pred = model(img_path)[0]

        pred_boxes = []
        for r in pred.boxes:
            cls = int(r.cls)
            xc, yc, w, h = r.xywhn[0]
            pred_boxes.append([cls, xc, yc, w, h])

        gt_xyxy = [yolo_to_xyxy(b, W, H) for b in gt_boxes]
        pred_xyxy = [yolo_to_xyxy(b, W, H) for b in pred_boxes]

        matched = set()
        for p in pred_xyxy:
            for i, g in enumerate(gt_xyxy):
                if iou(p, g) >= IOU_THRESH:
                    matched.add(i)

        if len(gt_boxes) > 0 and len(matched) == 0:
            category, out_dir = "missed", missed_dir
        elif 0 < len(matched) < len(gt_boxes):
            category, out_dir = "partial", partial_dir
        else:
            category, out_dir = "success", success_dir

        pred_panel = draw_boxes(
            img.copy(), pred_boxes, (0,0,255),
            f"{model_name} – {category.upper()}"
        )

        combined = np.hstack([gt_panel, pred_panel])
        cv2.imwrite(
            os.path.join(out_dir, f"{name}_{model_name}_{category}.jpg"),
            combined
        )

        panels.append(pred_panel)

    # ---------- ALL MODELS ----------
    max_h = max(p.shape[0] for p in panels)
    panels = [cv2.resize(p, (p.shape[1], max_h)) for p in panels]
    all_models = np.hstack(panels)
    cv2.imwrite(os.path.join(compare_dir, f"{name}_ALL_MODELS.jpg"), all_models)

    print("Processed:", name)

print("\n✔ DONE — segmentation-safe side-by-side analysis generated")


Loading: YOLOv8n-S3
Loading: YOLOv8s-S3
Loading: YOLOv8m-S3
Loading: YOLOv8l-S3
Loading: YOLOv8x-S3

image 1/1 C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test_glare_fixed\000101_jpg.rf.fbf75c64095342160f45a3f55d72e725.jpg: 320x320 6 trashs, 9.1ms
Speed: 1.5ms preprocess, 9.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test_glare_fixed\000101_jpg.rf.fbf75c64095342160f45a3f55d72e725.jpg: 320x320 5 trashs, 9.3ms
Speed: 0.0ms preprocess, 9.3ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test_glare_fixed\000101_jpg.rf.fbf75c64095342160f45a3f55d72e725.jpg: 320x320 5 trashs, 11.9ms
Speed: 1.5ms preprocess, 11.9ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 C:\Users\User\Documents\Rabab\dataset\Scenario 3\reflection\test_glare_fixed\000101_jpg.rf.fbf75c6409534216